In [1]:
from __future__ import print_function
from misc import *



In [3]:
"""DVS128 closed loop oscillator routine by Alexander Rivkind, Eldad Assa, Michael Kreiserman and Ehud Ahissar
based on: 
*DVS example by: Yuhuang Hu (duguyue100@gmail.com)
*"Syclop" paradigm: Ahissar and Assa 2017
"""
from reservoir import ESN
import numpy as np
import time
import SYCLOP_env as syc
import matplotlib.pyplot as plt


import cv2

import maestro
import teachers
from pyaer.dvs128 import DVS128
%matplotlib notebook

In [ ]:
class  Syclop_Teacher:
    def __init__(self):
        self.hp = HP()
        self.hp.dt = 0.1
        self.hp.omega0 = 2*np.pi / 10.
        self.hp.A = 5.0 #target amplitude of oscillations. Target max speed is hence omega*A
        self.hp.noi = 0.01
        self.speed_lim = self.hp.omega0 * self.hp.A
        self.vel = 0
    def step(self,dvs_strip):        
        dvs_strip = dvs_strip + self.hp.noi*np.random.normal(size=np.shape(dvs_strip))
        I = dvs_strip**2.0
        I = I/(np.sum(I)+1e-10)
        centre = np.sum(I*np.array(range(len(dvs_strip))))- len(dvs_strip)/2.0 
#         print('centre:',centre)
        self.vel += self.hp.dt*self.hp.omega0*centre 
        self.vel = self.vel if np.abs(self.vel)<self.speed_lim else self.speed_lim*np.sign(self.vel)

In [4]:
def kernel_weights_prep(n,m,w, kernel = None):
    if kernel is None:
        kernel = lambda x,mu: np.exp(-(x-mu)**2./(2.*w**2))/np.sqrt(np.pi)/w
    wk=np.zeros([n,m])
    for mm in range(m):
        v = np.linspace(0,m-1,n)
        wk[:,mm] = kernel(v,mm)
    return wk

In [5]:
servo = maestro.Controller()
esn = ESN(n_inputs = 128)
esn.hp.g=1.5
esn.reset()
esn.wfb = esn.wfb*1.0
esn.win = 5*kernel_weights_prep(esn.hp.N,128,5)
A_net_teacher = 5.0
fully_forced = 10

In [6]:
def gohome():    
    servo.setSpeed(0,0)
    servo.setSpeed(1,0)
    servo.setTarget(1,7500)     #set speed of servo 1
    servo.setTarget(0,5000)  #set servo to move to center position
    servo.setSpeed(0,1)
    servo.setSpeed(1,1)
    time.sleep(1.0)

In [ ]:
gohome()

In [ ]:
#teacher = Syclop_Teacher()

In [7]:
teacher = teachers.Clipped_Harmonic_1D()

In [11]:
training = False
device = DVS128()


device.start_data_stream()
# load new config
# device.set_bias_from_json("./scripts/configs/dvs128_config.json")
# print (device.get_bias())

clip_value = 3
histrange = [(0, v) for v in (128, 128)]
t_prev = time.time()



dt_nom = 0.025
# dt_list = []
cnt = 0
empty_cnt = 0

gohome()
while True:
    t_this = time.time()
    try:
        (pol_events, num_pol_event,
         special_events, num_special_event) = \
            device.get_event("events_hist")
        if t_this - t_prev > dt_nom:
                if num_pol_event != 0:
                    img = pol_events[..., 1]-pol_events[..., 0]
                    observation=np.sum(img,axis=0)**2
                    esn.step(uin = observation)
                    if training:
                        teacher.step(observation)
                        esn.FORCE_step(teacher.vel/A_net_teacher)
                    if cnt > fully_forced:
                        servo.moveByVelocity(0,(int(np.round(A_net_teacher*esn.z))))
                    else:
                        print('full force:', end='')
                        servo.moveByVelocity(0,(int(np.round(teacher.vel))))
                    img = np.clip(img, -clip_value, clip_value)
                    img = img+clip_value
                    print(cnt,'/',empty_cnt,'\r',end='')
                    cv2.imshow("image", img/float(clip_value*2))
                    t_prev=t_this
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    cnt += 1
        else:
            empty_cnt += 1

    except KeyboardInterrupt:
            device.shutdown()
            break


In [ ]:
cnt, empty_cnt

In [9]:
import pickle

In [10]:
with open('deleteme6.pkl','wb') as f:
    pickle.dump(esn,f)

In [ ]:
np.sum(esn.wout*esn.wfb)

In [ ]:
np.max(np.imag(np.linalg.eig(esn.W)[0]))

In [ ]:
import pickle

In [ ]:
with open('deleteme4.pkl','rb') as f:
    esn = pickle.load(f)